In [1]:
from tabula import read_pdf
import datetime
import pandas as pd

import warnings
warnings.simplefilter("ignore")

In [2]:
def extract_text(l):
    return "".join([k for k in l if not k.isdigit() and not k in [".", ",", "+"]]).rstrip()


def extract_numbers(l, el=None):
#     lis = "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ")
#     print(lis)
#     print([m for m in lis if len(m) > 0])
    result = [int(m) for m in "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ") if len(m) > 0]
    if len(result) < 4:
        result += [0]*(4-len(result))
    if el is None:
        return result
    else:
        return result[el]


def count_numbers(l):
    c = 0
    try:
        c += sum([1 for k in l if k.isdigit()])
        return c
    except:
        return c


def extract_number(l):
    result = "".join([k for k in l.values[0] if k.isdigit()]).lstrip().rstrip()
    if len(result) > 0:
        return int(result)
    else:
        return 0
# df.loc[:, ['confirmed']].apply(extract_number, axis=1)

In [220]:
def fix_misaligned_row(df):
    df = df.reset_index(drop=True)
    for row in df.itertuples():
        try:
            if row.land != 'nan' and "Mecklenburg" not in row.land and pd.isnull(sum(row[2:])) == True:
                prev_index = row[0]-1
                df.loc[prev_index, 'land'] = (str(df.loc[prev_index, 'land']).replace('nan', '') + ' ' + str(row.land).replace('nan', '')).strip()
            if "Mecklenburg" in row.land or "Pomerania" in row.land:
                df.loc[df.land == row.land, 'land'] = 'Mecklenburg-Western Pomerania'
        except:
            pass
    return df


def load_pdf(date, path="RKI-reports/{0}-{1}.pdf", lang="en"):
    from tabula import read_pdf
    path = path.format(date, lang)
    print(path)
    df = read_pdf(path, pages=[2])
    df = df[0].iloc[4:]
    try:
        print("Extracting single data column")
        df.columns = ['data']
        df['c'] = df.loc[:, 'data'].apply(count_numbers)
        df = fix_misaligned_row(df)
        df = df[df.c != 0]
        df['land'] = df.loc[:, 'data'].apply(extract_text)
        df['confirmed'] = df.loc[:, 'data'].apply(extract_numbers, args=[0])
        df['daily'] = df.loc[:, 'data'].apply(extract_numbers, args=[1])
        df['per_mil'] = df.loc[:, 'data'].apply(extract_numbers, args=[2])
        df['dead'] = df.loc[:, 'data'].apply(extract_numbers, args=[3])
    except:
        if len(df.columns) == 4:
            print("Extracting 4 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'dead']
            df['per_mil'] = 0
            df = df.loc[:, ['land', 'confirmed', 'daily', 'per_mil', 'dead']]
        elif len(df.columns) == 5:
            print("Extracting 5 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'per_mil', 'dead']
        elif len(df.columns) == 6:
            print("Extracting 6 data columns")
            df.columns = ['land', 'confirmed', 'daily', 'per_mil', 'dead', 'dead_per_100k']
            print(df.head(20))
        else:
            print(f"Falied to exctract {len(df.columns)} data columns")
            print(df)
            raise Exception
        df = fix_misaligned_row(df)
        df.drop(['dead_per_100k'], axis=1, inplace=True)
        df['c'] = df.loc[:, 'confirmed'].apply(count_numbers)
        df = df.loc[df.c != 0, :]
        df.loc[:,['confirmed']] = df.loc[:,['confirmed']].apply(extract_number, axis=1)
        df.loc[:,['daily']] = df.loc[:,['daily']].apply(extract_number, axis=1)
        df.loc[:,['dead']] = df.loc[:,['dead']].apply(extract_number, axis=1)
    df.loc[df.land.isnull() == True, 'land'] = 'Mecklenburg-Western Pomerania'
    df = df.loc[(df.land.str.contains('cases') == False) & (df.land != 'Total') 
                & (df.land.str.contains('Gesamt') == False), :]
    try:
        df['date'] = datetime.datetime.strptime(date, "%Y-%m-%d")
    except:
        df['date'] = date
    df.drop('c', axis=1, inplace=True)
    return df.reset_index(drop=True)

# Test Load

In [221]:
df = load_pdf('2020-05-11', lang="en")
df

RKI-reports/2020-05-11-en.pdf
Extracting single data column
Extracting 6 data columns
                      land confirmed daily per_mil   dead dead_per_100k
4      Baden-Wuerttemberg*    33,190   -97     300  1,545          14.0
5                  Bavaria    44,368   103     339  2,155          16.5
6                   Berlin     6,272    11     167    165           4.4
7              Brandenburg     3,106     5     124    134           5.3
8                   Bremen     1,055    12     154     34           5.0
9                  Hamburg     4,780     8     260    204          11.1
10                   Hesse     9,012    14     144    412           6.6
11     Mecklenburg-Western       NaN   NaN     NaN    NaN           NaN
12                     NaN       728     2      45     19           1.2
13               Pomerania       NaN   NaN     NaN    NaN           NaN
14            Lower Saxony    10,854    27     136    498           6.2
15  North Rhine-Westphalia    35,132   168     196

,land,confirmed,daily,per_mil,dead,date
0,Baden-Wuerttemberg*,33190,97,300,1545,2020-05-11
1,Bavaria,44368,103,339,2155,2020-05-11
2,Berlin,6272,11,167,165,2020-05-11
3,Brandenburg,3106,5,124,134,2020-05-11
4,Bremen,1055,12,154,34,2020-05-11
5,Hamburg,4780,8,260,204,2020-05-11
6,Hesse,9012,14,144,412,2020-05-11
7,Mecklenburg-Western Pomerania,728,2,45,19,2020-05-11
8,Lower Saxony,10854,27,136,498,2020-05-11
9,North Rhine-Westphalia,35132,168,196,1437,2020-05-11


# Batch Load

In [187]:
# df = pd.DataFrame(columns=['data', 'land', 'confirmed', 'daily', 'per_mil', 'dead'])
# list_df = []
# start='2020-03-18'
# end='2020-04-17'
# for d in pd.date_range(start, end).date:
#     list_df.append(load_pdf(d, path="data/RKI-reports/{0}-{1}.pdf", lang="en"))

## Batch Save

In [188]:
# result_df = pd.concat(list_df).drop(['daily', 'c', 'per_mil'], axis=1)
# result_df.to_csv('data/RKI-reports.csv', index=False)

-----

-----

# Updating Reports

In [274]:
df_new = load_pdf('2020-05-15', lang="en")
df_new.drop(['daily', 'per_mil'], axis=1, inplace=True)

RKI-reports/2020-05-15-en.pdf
Extracting single data column
Extracting 6 data columns
                      land confirmed daily per_mil   dead dead_per_100k
4       Baden-Wuerttemberg    33,851   181     306  1,628          14.7
5                  Bavaria    45,143   163     345  2,260          17.3
6                   Berlin     6,397    55     171    177           4.7
7              Brandenburg     3,158    23     126    148           5.9
8                   Bremen     1,129    25     165     37           5.4
9                  Hamburg     4,981     5     271    228          12.4
10                   Hesse     9,204    35     147    429           6.8
11     Mecklenburg-Western       NaN   NaN     NaN    NaN           NaN
12                     NaN       740     1      46     20           1.2
13               Pomerania       NaN   NaN     NaN    NaN           NaN
14            Lower Saxony    11,087    84     139    534           6.7
15  North Rhine-Westphalia    35,967   226     201

In [275]:
df_new = df_new[df_new.land != 'Federal State Total Number Number of Cases/ Number of'].drop_duplicates()

df_new.loc[df_new['land'].str.contains('Schleswig-Holstein') == True, 'land'] = 'Schleswig-Holstein'
df_new.loc[df_new['land'].str.contains('Baden-W') == True, 'land'] = 'Baden-Wuerttemberg'
df_new.loc[df_new['land'].str.contains('Saxony-A') == True, 'land'] = 'Saxony-Anhalt'
df_new.loc[df_new['land'].str.contains('Sachsen-Anhalt') == True, 'land'] = 'Saxony-Anhalt'
df_new.loc[df_new['land'].str.contains('Mecklenburg-W') == True, 'land'] = 'Mecklenburg-Western Pomerania'
df_new.loc[df_new['land'].str.contains('Mecklenburg-V') == True, 'land'] = 'Mecklenburg-Western Pomerania'


df_new.loc[df_new['land'].str.contains('Nordrhein-Westfalen') == True, 'land'] = 'North Rhine-Westphalia'
df_new.loc[df_new['land'].str.contains('Sachsen') == True, 'land'] = 'Saxony'
df_new.loc[df_new['land'].str.contains('Rheinland-Pfalz') == True, 'land'] = 'Rhineland-Palatinate'
df_new.loc[df_new['land'].str.contains('Niedersachsen') == True, 'land'] = 'Lower Saxony'
df_new.loc[df_new['land'].str.contains('Hessen') == True, 'land'] = 'Hesse'
df_new.loc[df_new['land'].str.contains('Thüringen') == True, 'land'] = 'Thuringia'

In [276]:
df_new

,land,confirmed,dead,date
0,Baden-Wuerttemberg,33851,1628,2020-05-15
1,Bavaria,45143,2260,2020-05-15
2,Berlin,6397,177,2020-05-15
3,Brandenburg,3158,148,2020-05-15
4,Bremen,1129,37,2020-05-15
5,Hamburg,4981,228,2020-05-15
6,Hesse,9204,429,2020-05-15
7,Mecklenburg-Western Pomerania,740,20,2020-05-15
8,Lower Saxony,11087,534,2020-05-15
9,North Rhine-Westphalia,35967,1493,2020-05-15


In [277]:
df_new.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False)

,confirmed
land,
Baden-Wuerttemberg,1
Bavaria,1
Berlin,1
Brandenburg,1
Bremen,1
Hamburg,1
Hesse,1
Lower Saxony,1
Mecklenburg-Western Pomerania,1


In [288]:
result_all = pd.read_csv('RKI-reports.csv')
result_all['date'] = result_all['date'].astype('datetime64[ns]')

In [289]:
# result_all.loc[~((result_all.land == 'Mecklenburg-Western Pomerania') & (result_all.confirmed > 43000)), :]

In [290]:
print(result_all.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50))

                               confirmed
land                                    
Baden-Wuerttemberg                    58
Bavaria                               58
Berlin                                58
Brandenburg                           58
Bremen                                58
Hamburg                               58
Hesse                                 58
Lower Saxony                          58
Mecklenburg-Western Pomerania         58
North Rhine-Westphalia                58
Rhineland-Palatinate                  58
Saarland                              58
Saxony                                58
Saxony-Anhalt                         58
Schleswig-Holstein                    58
Thuringia                             58


In [291]:
result_concat = pd.concat([result_all, df_new]).sort_values('date', ascending=False)
print(result_concat.head(2))

                             land  confirmed  dead       date data
15                      Thuringia       2681   135 2020-05-15  NaN
7   Mecklenburg-Western Pomerania        740    20 2020-05-15  NaN


In [292]:
result_all.date.max()

Timestamp('2020-05-14 00:00:00')

In [293]:
result_concat.date.max()

Timestamp('2020-05-15 00:00:00')

In [282]:
print(len(result_concat))
print(result_concat.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50))

949
                               confirmed
land                                    
Baden-Wuerttemberg                    59
Bavaria                               59
Berlin                                59
Brandenburg                           59
Bremen                                59
Hamburg                               59
Hesse                                 59
Lower Saxony                          59
Mecklenburg-Western Pomerania         59
North Rhine-Westphalia                59
Rhineland-Palatinate                  59
Saarland                              59
Saxony                                59
Saxony-Anhalt                         59
Schleswig-Holstein                    59
Thuringia                             59


In [283]:
# result_concat = result_concat.loc[~((result_concat.land == 'Saxony') & (result_concat.date=='2020-03-30') & (result_concat.data.str.contains('Anhalt') == True)), :]

In [284]:
# Drop Duplicates
result_concat = result_concat.drop_duplicates()

In [285]:
print(len(result_concat))
print(result_concat.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50))

949
                               confirmed
land                                    
Baden-Wuerttemberg                    59
Bavaria                               59
Berlin                                59
Brandenburg                           59
Bremen                                59
Hamburg                               59
Hesse                                 59
Lower Saxony                          59
Mecklenburg-Western Pomerania         59
North Rhine-Westphalia                59
Rhineland-Palatinate                  59
Saarland                              59
Saxony                                59
Saxony-Anhalt                         59
Schleswig-Holstein                    59
Thuringia                             59


### Save Concantinated Result

In [273]:
result_concat.to_csv('RKI-reports.csv', index=False)

---

In [185]:
# Testing for Duplicates
print(result_concat.loc[:, ['land', 'date', 'confirmed']].groupby(['land', 'date']).count().sort_values(['confirmed'], ascending=False).head())

                                 confirmed
land                 date                 
Baden-Wuerttemberg   2020-03-18          1
Rhineland-Palatinate 2020-04-27          1
                     2020-04-16          1
                     2020-04-17          1
                     2020-04-18          1


---

# Add Data Manually

In [193]:
df_in = result_concat.copy()
df_in.set_index('date', inplace=True)

df_extra_dates = pd.DataFrame(
    {'day': pd.Series([min(df_in.index) , max(df_in.index)])})
df_extra_dates.set_index('day', inplace=True)
df_extra_dates = df_extra_dates.asfreq('D')

for d in df_extra_dates.index:
    for l in df_in.land.unique():
        if len(str(l)) > 3:
            try: 
                df_in.loc[(df_in.index == d) & (df_in.land==l), 'confirmed'].values[0]
            except:
                print(l, d)

Baden-Wuerttemberg 2020-03-18 00:00:00
Mecklenburg-Western Pomerania 2020-03-23 00:00:00
Mecklenburg-Western Pomerania 2020-03-25 00:00:00
Mecklenburg-Western Pomerania 2020-03-27 00:00:00
Mecklenburg-Western Pomerania 2020-03-28 00:00:00
Mecklenburg-Western Pomerania 2020-03-29 00:00:00
Saxony-Anhalt 2020-03-30 00:00:00
Berlin 2020-04-27 00:00:00
Bavaria 2020-04-27 00:00:00
Baden-Wuerttemberg 2020-04-27 00:00:00
Bavaria 2020-04-28 00:00:00
Baden-Wuerttemberg 2020-04-28 00:00:00
Bavaria 2020-04-29 00:00:00
Baden-Wuerttemberg 2020-04-29 00:00:00
Bavaria 2020-05-03 00:00:00
Baden-Wuerttemberg 2020-05-03 00:00:00


----

In [175]:
columns = ['land','confirmed','dead','date','data']
new_data = {"Berlin": [
    [5638, 125, datetime.datetime(2020, 4, 27), ''],
],
"Saxony-Anhalt": [
    [592, 2, datetime.datetime(2020, 3, 30), ''],
],
"Bavaria": [
    [41070, 1621, datetime.datetime(2020, 4, 27), ''],
    [41406, 1681, datetime.datetime(2020, 4, 28), ''],
    [41830, 1754, datetime.datetime(2020, 4, 29), ''],
    [42792, 1910, datetime.datetime(2020, 5, 3), ''],
],
"Baden-Wuerttemberg": [
    [1609, 2, datetime.datetime(2020, 3, 18), ''],
    [31043, 1249, datetime.datetime(2020, 4, 27), ''],
    [31196, 1269, datetime.datetime(2020, 4, 28), ''],
    [31336, 1299, datetime.datetime(2020, 4, 29), ''],
    [32291, 1412, datetime.datetime(2020, 5, 3), ''],
    
],
"Mecklenburg-Western Pomerania": [
    [172, 0, datetime.datetime(2020, 3, 23), ''],
    [218, 0, datetime.datetime(2020, 3, 25), ''],
    [259, 0, datetime.datetime(2020, 3, 27), ''],
    [308, 0, datetime.datetime(2020, 3, 28), ''],
    [348, 1, datetime.datetime(2020, 3, 29), ''],
],}

In [196]:
df_l = []
for k in new_data:
    for d in new_data[k]:
        dic = {'land': k, 'confirmed':d[0],'dead':d[1],'date':d[2],'data':''}
        df_l.append(dic)

df_new_data = pd.DataFrame(df_l)


In [197]:
result_concat2 = pd.concat([result_concat, df_new_data]).sort_values('date', ascending=False)

In [31]:
print(result_concat2.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50))

                               confirmed
land                                    
Baden-Wuerttemberg                    47
Bavaria                               47
Berlin                                47
Brandenburg                           47
Bremen                                47
Hamburg                               47
Hesse                                 47
Lower Saxony                          47
Mecklenburg-Western Pomerania         47
North Rhine-Westphalia                47
Rhineland-Palatinate                  47
Saarland                              47
Saxony                                47
Saxony-Anhalt                         47
Schleswig-Holstein                    47
Thuringia                             47


In [200]:
# result_concat2.to_csv('data/RKI-reports.csv', index=False)

---